In [2]:
import pandas as pd
import numpy as np

In [5]:
news_df = pd.read_csv('./data_download/news.csv')
news_df = news_df[['title', 'text', 'topic']]
news_df.head()

,title,text,topic
0,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия
1,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия
2,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура
3,Полицейские застрелили порезавшего мать буйног...,В Москве полицейские застрелили мужчину при по...,Силовые структуры
4,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир


In [7]:
news_df['topic'].value_counts()

Россия               10996
Мир                   6457
Экономика             4792
Бывший СССР           3798
Спорт                 3563
Наука и техника       2828
Силовые структуры     2714
Интернет и СМИ        2623
Из жизни              2432
Культура              2339
Путешествия           2029
Ценности              1636
Среда обитания        1484
Нацпроекты            1450
69-я параллель         280
Дом                    254
Оружие                   2
Бизнес                   2
Name: topic, dtype: int64

In [9]:
news_filtered_df = news_df[news_df['topic'].isin(['Культура', 'Экономика', 'Спорт', 'Наука и техника', 'Интернет и СМИ'])]
news_filtered_df.to_csv('./data_download/news_filtered.csv')